In [1]:
#  Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

from tabulate import tabulate

from gams import *

In [3]:
# Load the gams_magic, run tutorial notebook for function definition

%load_ext gams_magic
%gams_reset
%gams_cleanup -k

The gams_magic extension is already loaded. To reload it, use:
  %reload_ext gams_magic


In [3]:
# Aux functions from the DataTransform notebook

def gt_from2dim(df, column_names=None):
    if column_names==None:
        return pd.DataFrame(df.stack()).reset_index()
    else:
        df = pd.DataFrame(df.stack()).reset_index()
        return df.rename(columns=dict(zip(df.columns, column_names)))

def gt_pivot(df, index=None, columns=None, values=None):
    df = df.pivot(index=index, columns=columns, values=values)
    df.index.names = [None]
    df.columns.names = [None]
    return df

def gt_pivot2d(df):
    return gt_pivot(df,index=df.columns[0], columns=df.columns[1], values=df.columns[2])

In [4]:
# loadmat aux function to load the matlab save files
# from: https://stackoverflow.com/review/suggested-edits/21667510

import scipy.io as spio

def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    def _check_keys(d):
        '''
        checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        '''
        for key in d:
            if isinstance(d[key], spio.matlab.mio5_params.mat_struct):
                d[key] = _todict(d[key])
        return d

    def _has_struct(elem):
        """Determine if elem is an array and if any array item is a struct"""
        return isinstance(elem, np.ndarray) and any(isinstance(
                    e, scipy.io.matlab.mio5_params.mat_struct) for e in elem)

    def _todict(matobj):
        '''
        A recursive function which constructs from matobjects nested dictionaries
        '''
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, spio.matlab.mio5_params.mat_struct):
                d[strg] = _todict(elem)
            elif _has_struct(elem):
                d[strg] = _tolist(elem)
            else:
                d[strg] = elem
        return d

    def _tolist(ndarray):
        '''
        A recursive function which constructs lists from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        '''
        elem_list = []
        for sub_elem in ndarray:
            if isinstance(sub_elem, spio.matlab.mio5_params.mat_struct):
                elem_list.append(_todict(sub_elem))
            elif _has_struct(sub_elem):
                elem_list.append(_tolist(sub_elem))
            else:
                elem_list.append(sub_elem)
        return elem_list
    data = scipy.io.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

In [5]:
# Matlab code conversion

# Read matlab file instead of excelfile (easier conversion)
data = loadmat('matlab.mat')
data.keys()

C:\Users\camar\AppData\Local\Temp\ipykernel_14604\3560610281.py:19: DeprecationWarning: Please use `mat_struct` from the `scipy.io.matlab` namespace, the `scipy.io.matlab.mio5_params` namespace is deprecated.
  if isinstance(d[key], spio.matlab.mio5_params.mat_struct):
C:\Users\camar\AppData\Local\Temp\ipykernel_14604\3560610281.py:35: DeprecationWarning: Please use `mat_struct` from the `scipy.io.matlab` namespace, the `scipy.io.matlab.mio5_params` namespace is deprecated.
  if isinstance(elem, spio.matlab.mio5_params.mat_struct):
C:\Users\camar\AppData\Local\Temp\ipykernel_14604\3560610281.py:26: DeprecationWarning: Please use `mat_struct` from the `scipy.io.matlab` namespace, the `scipy.io.matlab.mio5_params` namespace is deprecated.
  e, scipy.io.matlab.mio5_params.mat_struct) for e in elem)


dict_keys(['__header__', '__version__', '__globals__', 'data'])

In [15]:
# Linearize the generator prices

# genCofA=data.generator.limit(:,:,6); -> MATLAB starts with 1, so we need the index -1
# genCofB=data.generator.limit(:,:,7);
# genCofC=data.generator.limit(:,:,8);

genCofA = data['data']['generator']['limit'][:, :, 5]
genCofB = data['data']['generator']['limit'][:, :, 6]
genCofC = data['data']['generator']['limit'][:, :, 7]

In [19]:
# Connection with GAMS

%gams option iterlim = 1000000000;
%gams option reslim = 5000000000;

## Cell to define GAMS variables

In [20]:
%%gams

set genLimitInfo 'Indicate the number of columns of information from generators'
/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

set loadLimitInfo 'Indicate the number of columns of information from loads'
/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

set storLimitInfo 'Indicate the number of columns of information from storage units'
/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

set v2gLimitInfo 'Indicate the number of columns of information from electric vehicles'
/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

set csLimitInfo 'Indicate the number of columns of information from charging stations'
/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

## Push Python variables to GAMS

In [ ]:
# TODO


set t /1*%period%/;

set gen /1*%gen%/;

set load /1*%Load%/;

set stor /1*%stor%/;

set v2g /1*%v2g%/;

set cs /1*%cs%/;